In [1]:
## 경고 문구 발생시 command + shift + p => reload window
!pip list
!which python  # Linux/Mac

# source .venv/bin/activate

Package                   Version
------------------------- --------------
ace_tools                 0.0
anyio                     4.7.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
args                      0.1.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     24.3.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.2.0
certifi                   2024.12.14
cffi                      1.17.1
charset-normalizer        3.4.1
clint                     0.5.1
comm                      0.2.2
coverage                  7.6.10
debugpy                   1.8.11
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.1.0
fastjsonschema            2.21.1
fqdn                      1.5.1
h11                       0.14.0
httpcore                  1.0.7
httpx                     0.28.1
idna           

## 크롬 드라이버 프로세스 강제 종료

In [3]:
import psutil

# 종료하려는 프로세스 이름
process_name = "chrome"  # 또는 "Google Chrome" (운영체제에 따라 다를 수 있음)

# 모든 프로세스를 조회하여 이름이 'chrome'인 프로세스를 종료
for proc in psutil.process_iter(['pid', 'name']):
    try:
        # 프로세스 이름이 chrome인지 확인
        if process_name.lower() in proc.info['name'].lower():
            print(f"Terminating process {proc.info['name']} with PID {proc.info['pid']}")
            proc.terminate()  # 프로세스 종료
            proc.wait()  # 종료가 완료될 때까지 대기
            print(f"Process {proc.info['name']} terminated successfully")
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass  # 프로세스가 이미 종료되었거나 권한이 없을 경우 예외 처리


## 셀레니움 세팅

In [10]:
## reference - https://thecho7.tistory.com/entry/Python과-Selenium을-활용한-웹-크롤링-실습-1

# 느낌표(!)는 Jupyter에서 패키지를 설치할 때 필요한 커맨드입니다
!pip install selenium

# 우리는 Chrome을 사용할건데, 이 패키지를 설치하면 버전 관리를 손쉽게 할 수 있습니다
# cmd + shift + p를 통해 reload windows를 하면 코드 경고 사라집니다
# https://github.com/SergeyPirogov/webdriver_manager
!pip install webdriver-manager

# 데이터 출력을 위한 라이브러리
!pip install pandas

## import

In [4]:
# 작업에 필요한 패키지를 불러옵니다
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# 데이터 출력
import pandas as pd

# Chrome 브라우저를 오픈합니다
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.get("https://www.naver.com")


## 크롤링 최적화

In [5]:

## 드라이버 최적화
def driver_Settings():
    options = Options()

    # 웹드라이버 종료 안시키고 유지
    #options.add_experimental_option("detach", True)

    # 주석해제하면 헤드리스 모드
    #options.add_argument("--headless")  # 헤드리스 모드로 실행

    # Windows 10 운영 체제에서 Chrome 브라우저를 사용하는 것처럼 보이는 사용자 에이전트가 설정
    options.add_argument(
        'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

    # 언어 설정을 한국어로 지정. 이는 웹 페이지가 한국어로 표시
    options.add_argument('lang=ko_KR')

    # 브라우저 창의 크기를 지정. 여기서는 너비 430px, 높이 932px로 설정
    options.add_argument('--window-size=932,932')

    # GPU 가속을 비활성화. GPU 가속이 활성화되어 있으면, Chrome이 GPU를 사용하여 그래픽을 렌더링하려고 시도할 수 있기때문. 일부 환경에서는 GPU 가속이 문제를 일으킬 수 있으므로 이 옵션을 사용하여 비활성화
    options.add_argument('--disable-gpu')

    # 정보 표시줄을 비활성화. 정보 표시줄은 Chrome 브라우저 상단에 나타나는 알림이나 메시지를 의미. 이 옵션을 사용하여 이러한 알림이 나타나지 않도록 설정.
    options.add_argument('--disable-infobars')

    # 확장 프로그램을 비활성화. Chrome에서 확장 프로그램을 사용하지 않도록 설정
    options.add_argument('--disable-extensions')

    #  자동화된 기능을 비활성화. 이 옵션은 Chrome이 자동화된 환경에서 실행되는 것을 감지하는 것을 방지
    options.add_argument('--disable-blink-features=AutomationControlled')

    # 자동화를 비활성화. 이 옵션은 Chrome이 자동화 도구에 의해 제어되는 것으로 감지되는 것을 방지
    options.add_argument('--disable-automation')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    return driver

In [6]:
driver = driver_Settings()



In [7]:

page_nums = 5
search = "아이폰"

# 데이터를 저장할 리스트
data = []

# 0, 10, 20, 30
for page_num in range(0, page_nums*10, 10):
    driver.get(f"https://www.google.com/search?q={search}&sca_esv=1f2f2e42c8b63453&tbm=nws&sxsrf=ADLYWIJiMdOV1vEJqiO9AKtDvtz5QmvpWA:1735375337578&ei=6blvZ473Itu3vr0P08HGyA0&start={page_num}&sa=N&ved=2ahUKEwjOj7H0iMqKAxXbm68BHdOgEdk4FBDy0wN6BAgEEAQ&biw=933&bih=973&dpr=1.8")
    #rso > div > div > div
    posts = driver.find_elements(By.CSS_SELECTOR, "#rso > div > div > div")


    for post in posts:
        post_info = post.find_elements(By.CSS_SELECTOR, "div > div > a > div > div:nth-child(2) > div")

        company = post_info[0].text
        title = post_info[1].text
        content = post_info[2].text
        time = post_info[-1].text
        post_url = post.find_element(By.CSS_SELECTOR, "div > div > a").get_attribute("href")
        
        

        data.append([company, title, content, time, post_url])
        

        # print(f"""
        # {company}
        # 제목: {title}
        # 내용: {content}
        # 시간: {time}
        # """)
# 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(data, columns=["Company", "Title", "Content", "Time", "Link"])

# pandas 옵션 설정: 생략 없이 모든 행과 열을 출력
pd.set_option('display.max_rows', None)  # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.width', None)  # 출력 너비 제한 해제
pd.set_option('display.max_colwidth', None)  # 열 너비 제한 해제
display(df)



,Company,Title,Content,Time,Link
0,디지털투데이,"차세대 아이폰17 프로, 다이내믹 아일랜드 크기 작아진다",[디지털투데이 AI리포터] 차세대 애플 아이폰17 프로에 대한 새로운 소문이 나왔다. 26일(현지시간) 모바일 매체 폰아레나에 따르면 아이폰17 프로의...,2일 전,https://www.digitaltoday.co.kr/news/articleView.html?idxno=547454
1,지디넷코리아,"아이폰17 프로 렌더링, 중국서 나왔다…달라진 점은",애플이 내년 가을 출시할 예정인 아이폰17 프로의 모습이 담긴 렌더링이 공개됐다. 중국 IT매체 마이드라이버스는 여러 소식통을 통해 애플이 메타...,2일 전,https://zdnet.co.kr/view/?no=20241227105839
2,ZUM 뉴스,“아이폰 타령 너무 심하다했더니” 20대 요금 연체 속출…결국 ‘초유의 일’ 터졌다,아이폰 매장 [헤럴드경제= 박영훈 기자] “너무 비싸 이제 휴대폰 못 바꾼다?” 아이폰 등 비싼 스마트폰을 감당하지 못해 요금을 연체하거나 미납한 20...,17시간 전,https://news.zum.com/articles/95411403
3,IT조선,"오드엔티, ‘게트리 플랫 아이폰16 마그네틱 케이스’ 출시",글로벌 크리에이터 잇섭의 글로벌 테크 기업 오드엔티가 '게트리 플랫 아이폰16 마그네틱 케이스'를 출시했다고 27일 밝혔다. 게트리 플랫 아이폰16...,2일 전,https://it.chosun.com/news/articleView.html?idxno=2023092130825
4,NATE,"""아차!"" 아이폰 헌금함에 떨어뜨리자 ""이미 신의 것"" 반환 거부한 절",아이폰을 실수로 헌금함에 떨어뜨렸다가 돌려 받지 못한 인도 청년의 사연이 화제다. 지난 28일 MGA온라인 등 외신에 따르면 최근 인도 청년 A씨는...,5시간 전,https://news.nate.com/view/20241229n02889
5,매일신문,불붙은 두께 경쟁…갤럭시·아이폰 슬림 모델 내년 출시?,성장 정체기를 맞은 스마트폰 시장이 내년에는 '두께 경쟁'으로 새로운 국면을 맞을 것이란 전망이 나온다. 일부 중국 기업들이 이미 얇은 모델을...,1일 전,https://www.imaeil.com/page/view/2024122810554833077
6,세계일보,"실수로 헌금함에 아이폰 빠졌는데…印 ""이미 신의 소유물""","최근 인도의 한 남성이 사원 헌금함에 실수로 빠뜨린 휴대전화를 돌려달라고 요청했으나, 사원 측이 ""이미 신의 소유물""이라며 거부해 논란이 일고...",2일 전,https://www.segye.com/newsView/20241227500429
7,이뉴스투데이,‘폴더블 스마트폰’ 인기 시들한데···‘접는’ 아이폰 출시 괜찮을까,[이뉴스투데이 고선호 기자] 애플의 '폴더블' 아이폰 출시가 본격화되면서 삼성전자 중심의 폴더블 스마트폰 시장에 지각변동이 불가피할 것으로 예상...,2일 전,http://www.enewstoday.co.kr/news/articleView.html?idxno=2218111
8,지디넷코리아,"""아이폰18 프로, 가변 조리개 카메라 시스템 탑재""",애플이 2026년 출시할 예정인 아이폰18 프로에 아이폰 최초로 가변 조리개 광각 렌즈를 탑재할 것이라는 전망이 나왔다. IT매체 애플인사이더는 최근...,3일 전,https://zdnet.co.kr/view/?no=20241226104210
9,지디넷코리아,아이폰17 에어가 아이폰17 프로보다 좋은 이유 4가지,"애플이 내년 가을 출시할 아이폰17 시리즈에 새롭게 '아이폰17 에어'가 추가된다는 전망이 나오면서, 아이폰17 에어에 대한 관심이 높아지고 있다.",3일 전,https://zdnet.co.kr/view/?no=20241226092053


In [41]:
from datetime import datetime, timedelta
    
# 시간 데이터를 datetime 형식으로 변환
def parse_time(time_str):
    if "시간 전" in time_str:
        hours_ago = int(time_str.replace("시간 전", "").strip())
        return datetime.now() - timedelta(hours=hours_ago)
    elif "일 전" in time_str:
        days_ago = int(time_str.replace("일 전", "").strip())
        return datetime.now() - timedelta(days=days_ago)
    elif "주 전" in time_str:
        weeks_ago = int(time_str.replace("주 전", "").strip())
        return datetime.now() - timedelta(weeks=weeks_ago)
    elif "분 전" in time_str:
        minutes_ago = int(time_str.replace("분 전", "").strip())
        return datetime.now() - timedelta(minutes=minutes_ago)
    else:
        return datetime.now()  # 기본적으로 현재 시간을 반환

# 'Time' 열을 실제 시간으로 변환
df["Time_Converted"] = df["Time"].apply(parse_time)

# 최신순으로 시간 정렬
df_sorted = df.sort_values(by="Time_Converted", ascending=False)

# pandas 옵션 설정: 생략 없이 모든 행과 열을 출력
pd.set_option('display.max_rows', None)  # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.width', None)  # 출력 너비 제한 해제
pd.set_option('display.max_colwidth', None)  # 열 너비 제한 해제

# 정렬된 데이터프레임 출력
display(df_sorted)



,Company,Title,Content,Time,Link,Time_Converted
19,지디넷코리아,새해 초 나올 애플 신제품은 무엇?,애플이 새해 초 강력한 신제품들을 대거 선보이며 2025년 한 해를 시작할 것으로 보인다. IT매체 나인투파이브맥은 애플이 내년 초 출시할 것으로...,6시간 전,https://zdnet.co.kr/view/?no=20241228065936,2024-12-28 12:31:38.174079
12,핀포인트뉴스,"LG이노텍, 아이폰 SE4 출시 기대에도 실적 부진으로 목표주가 하향",LG이노텍이 아이폰 SE4 출시 기대감에도 불구하고 단기적인 실적 부진으로 인해 목표주가가 하향 조정됐다.대신증권은 최근 보고서에서 전자 부품...,7시간 전,https://www.pinpointnews.co.kr/news/articleView.html?idxno=309553,2024-12-28 11:31:38.174057
10,뉴스저널리즘,"카카오페이손보, 아이폰 배터리 교체 보장 신설",카카오페이손해보험은 사용자 성원에 힘입어 휴대폰보험 상품을 개편한다고 27일 밝혔다. 이번 개편의 핵심은 휴대폰보험(2년 이내 출시한 갤럭시·...,8시간 전,https://www.ngetnews.com/news/articleView.html?idxno=517203,2024-12-28 10:31:38.174050
1,매일신문,불붙은 두께 경쟁…갤럭시·아이폰 슬림 모델 내년 출시?,성장 정체기를 맞은 스마트폰 시장이 내년에는 '두께 경쟁'으로 새로운 국면을 맞을 것이란 전망이 나온다. 일부 중국 기업들이 이미 얇은 모델을...,8시간 전,https://www.imaeil.com/page/view/2024122810554833077,2024-12-28 10:31:38.174017
18,디지털투데이,신제품 쏟아진다…2025년 초 애플 기대작 5가지,"[디지털투데이 홍진주 기자] 올해가 며칠 남지 않은 가운데 애플이 2025년부터 아이폰, 맥 제품군을 비롯한 하드웨어에 대대적인 업그레이드를 준비...",11시간 전,https://www.digitaltoday.co.kr/news/articleView.html?idxno=547490,2024-12-28 07:31:38.174075
14,헤럴드경제,"삼성D, 갤럭시로 노하우 쌓은 보람 있네…애플 첫 ‘이것’까지 꿰찼다 [비즈360]","[헤럴드경제=김민지 기자] 애플의 첫 폴더블 아이폰이 2026년 양산될 것으로 추정되는 가운데, 삼성디스플레이가 초기 폴더블 패널 물량을 독점 공급...",11시간 전,https://biz.heraldcorp.com/article/10032362,2024-12-28 07:31:38.174064
11,동아일보,"Qi2 무선 충전, ‘애플 아이폰은 15부터 지원··· 갤럭시도 S25부터 제한적 지원’",애플 맥세이프 기반의 새로운 무선 충전 프로필을 갖춘 Qi 2 무선충전이 내년부터 본격적으로 널리 쓰일 전망이다. Qi 무선충전 규격은 기(氣)의...,21시간 전,https://www.donga.com/news/It/article/all/20241227/130741636/1,2024-12-27 21:31:38.174054
46,에이빙,"애플, 2025년 초부터 신제품 릴레이 시작한다... M4 칩 탑재 맥북 에어 & 아이폰 SE 4세대 등 출격 대기!","애플이 오는 2025년부터 아이폰, 맥(Mac) PC 제품군을 비롯한 자사 하드웨어에 대대적인 업그레이드를 준비 중이라는 소식이 전해졌다.26일(현지...",1일 전,https://kr.aving.net/news/articleView.html?idxno=1796491,2024-12-27 18:31:38.174153
40,머니투데이,"애플, 사상 첫 시총 4조달러 눈 앞에…AI 아이폰이 상승 모멘텀",애플 주가가 26일(현지시간) 사상최고가를 경신하며 시가총액이 4조달러에 근접해 갔다. 애플 주가는 이날 0.3% 오른 259.02달러로 마감하며 5거래...,1일 전,https://news.mt.co.kr/mtview.php?no=2024122708432693852,2024-12-27 18:31:38.174136
39,뉴스1,"애플, 세계 최초 시총 4조 달러 근접…""AI 기능 강화된 아이폰 관심""(상보)",웨드부시증권 목표주가 상향…26% 추가 상승 여력 애플이 세계 최초로 시가총액 4조 달러를 달성하는 기업에 이름을 올릴 기세다.,1일 전,https://www.news1.kr/world/international-economy/5643709,2024-12-27 18:31:38.174134


In [40]:
from datetime import datetime, timedelta

# 시간 데이터를 datetime 형식으로 변환
def parse_time(time_str):
    if "시간 전" in time_str:
        hours_ago = int(time_str.replace("시간 전", "").strip())
        return datetime.now() - timedelta(hours=hours_ago)
    elif "일 전" in time_str:
        days_ago = int(time_str.replace("일 전", "").strip())
        return datetime.now() - timedelta(days=days_ago)
    elif "주 전" in time_str:
        weeks_ago = int(time_str.replace("주 전", "").strip())
        return datetime.now() - timedelta(weeks=weeks_ago)
    elif "분 전" in time_str:
        minutes_ago = int(time_str.replace("분 전", "").strip())
        return datetime.now() - timedelta(minutes=minutes_ago)
    else:
        return datetime.now()  # 기본적으로 현재 시간을 반환

# 'Time' 열을 실제 시간으로 변환
df["Time_Converted"] = df["Time"].apply(parse_time)

# 최신순으로 시간 정렬
df_sorted = df.sort_values(by="Time_Converted", ascending=False)

# 인덱스 재설정 (0부터 시작하도록)
df_sorted.reset_index(drop=True, inplace=True)

# pandas 옵션 설정: 생략 없이 모든 행과 열을 출력
pd.set_option('display.max_rows', None)  # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.width', None)  # 출력 너비 제한 해제
pd.set_option('display.max_colwidth', None)  # 열 너비 제한 해제

# 정렬된 데이터프레임 출력
display(df_sorted)


,Company,Title,Content,Time,Link,Time_Converted
0,지디넷코리아,새해 초 나올 애플 신제품은 무엇?,애플이 새해 초 강력한 신제품들을 대거 선보이며 2025년 한 해를 시작할 것으로 보인다. IT매체 나인투파이브맥은 애플이 내년 초 출시할 것으로...,6시간 전,https://zdnet.co.kr/view/?no=20241228065936,2024-12-28 12:31:09.190179
1,핀포인트뉴스,"LG이노텍, 아이폰 SE4 출시 기대에도 실적 부진으로 목표주가 하향",LG이노텍이 아이폰 SE4 출시 기대감에도 불구하고 단기적인 실적 부진으로 인해 목표주가가 하향 조정됐다.대신증권은 최근 보고서에서 전자 부품...,7시간 전,https://www.pinpointnews.co.kr/news/articleView.html?idxno=309553,2024-12-28 11:31:09.190161
2,뉴스저널리즘,"카카오페이손보, 아이폰 배터리 교체 보장 신설",카카오페이손해보험은 사용자 성원에 힘입어 휴대폰보험 상품을 개편한다고 27일 밝혔다. 이번 개편의 핵심은 휴대폰보험(2년 이내 출시한 갤럭시·...,8시간 전,https://www.ngetnews.com/news/articleView.html?idxno=517203,2024-12-28 10:31:09.190155
3,매일신문,불붙은 두께 경쟁…갤럭시·아이폰 슬림 모델 내년 출시?,성장 정체기를 맞은 스마트폰 시장이 내년에는 '두께 경쟁'으로 새로운 국면을 맞을 것이란 전망이 나온다. 일부 중국 기업들이 이미 얇은 모델을...,8시간 전,https://www.imaeil.com/page/view/2024122810554833077,2024-12-28 10:31:09.190127
4,디지털투데이,신제품 쏟아진다…2025년 초 애플 기대작 5가지,"[디지털투데이 홍진주 기자] 올해가 며칠 남지 않은 가운데 애플이 2025년부터 아이폰, 맥 제품군을 비롯한 하드웨어에 대대적인 업그레이드를 준비...",11시간 전,https://www.digitaltoday.co.kr/news/articleView.html?idxno=547490,2024-12-28 07:31:09.190176
5,헤럴드경제,"삼성D, 갤럭시로 노하우 쌓은 보람 있네…애플 첫 ‘이것’까지 꿰찼다 [비즈360]","[헤럴드경제=김민지 기자] 애플의 첫 폴더블 아이폰이 2026년 양산될 것으로 추정되는 가운데, 삼성디스플레이가 초기 폴더블 패널 물량을 독점 공급...",11시간 전,https://biz.heraldcorp.com/article/10032362,2024-12-28 07:31:09.190166
6,동아일보,"Qi2 무선 충전, ‘애플 아이폰은 15부터 지원··· 갤럭시도 S25부터 제한적 지원’",애플 맥세이프 기반의 새로운 무선 충전 프로필을 갖춘 Qi 2 무선충전이 내년부터 본격적으로 널리 쓰일 전망이다. Qi 무선충전 규격은 기(氣)의...,21시간 전,https://www.donga.com/news/It/article/all/20241227/130741636/1,2024-12-27 21:31:09.190158
7,에이빙,"애플, 2025년 초부터 신제품 릴레이 시작한다... M4 칩 탑재 맥북 에어 & 아이폰 SE 4세대 등 출격 대기!","애플이 오는 2025년부터 아이폰, 맥(Mac) PC 제품군을 비롯한 자사 하드웨어에 대대적인 업그레이드를 준비 중이라는 소식이 전해졌다.26일(현지...",1일 전,https://kr.aving.net/news/articleView.html?idxno=1796491,2024-12-27 18:31:09.190243
8,머니투데이,"애플, 사상 첫 시총 4조달러 눈 앞에…AI 아이폰이 상승 모멘텀",애플 주가가 26일(현지시간) 사상최고가를 경신하며 시가총액이 4조달러에 근접해 갔다. 애플 주가는 이날 0.3% 오른 259.02달러로 마감하며 5거래...,1일 전,https://news.mt.co.kr/mtview.php?no=2024122708432693852,2024-12-27 18:31:09.190229
9,뉴스1,"애플, 세계 최초 시총 4조 달러 근접…""AI 기능 강화된 아이폰 관심""(상보)",웨드부시증권 목표주가 상향…26% 추가 상승 여력 애플이 세계 최초로 시가총액 4조 달러를 달성하는 기업에 이름을 올릴 기세다.,1일 전,https://www.news1.kr/world/international-economy/5643709,2024-12-27 18:31:09.190226
